In [122]:
import csv
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
import re
import numpy as np
from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [186]:
import os
folder_path = 'stopwords'

# Get the list of files in the folder
file_list = os.listdir(folder_path)

all_words = []

# Read each text file and extract words from each line
for filename in file_list:
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            for line in file:
                words_in_line = line.strip().split()
                all_words.extend(words_in_line)

all_words = [word for word in all_words if '|' not in word]
all_words = [word for word in all_words if not word.startswith(('http', 'www.','Surnames'))]
all_words = [word for word in all_words if '(' not in word and ')' not in word]
# Print the list of all words
# print(all_words)


In [188]:
len(all_words)

14231

In [189]:
folder_path = 'MasterDictionary'



positive_words = []
negative_words = []

# Read words from the first file and store them in positive_list
file1_path = os.path.join(folder_path, 'positive-words.txt')
with open(file1_path, 'r') as file1:
    for line in file1:
        words = line.strip().split()
        positive_words.extend(words)

# Read words from the second file and store them in negative_list
file2_path = os.path.join(folder_path, 'negative-words.txt')
with open(file2_path, 'r') as file2:
    for line in file2:
        words = line.strip().split()
        negative_words.extend(words)

In [190]:
len(text_files)

111

In [191]:
print('length of positive and negative list is', len(positive_words),',',len(negative_words))

length of positive and negative list is 2006 , 4783


In [192]:
folder_name = 'scraped_data'

In [43]:
def pos_neg_pola_subj_avglen(filename):
    
#     master_dict ={}
    positive_list = []
    negative_list = []
    
    file_path = os.path.join(folder_name, filename)
    with open(file_path,'r',encoding='utf-8') as file:
        corpus = file.read()
        
    corpus = corpus.lower()
    
    corpus_for_avglen = re.sub('[^A-Za-z0-9.]+', ' ', corpus)
    sentences = nltk.sent_tokenize(corpus_for_avglen)

    corpus = re.sub('[^A-Za-z0-9]+', ' ', corpus)

    token_list = nltk.tokenize.word_tokenize(corpus)
    
    for word in token_list:
        if word in positive_words:
            positive_list.append(word)
        elif word in negative_words:
            negative_list.append(word)
        else:
            pass
    
    positive_score = len(positive_list)
    negative_score = len(negative_list)
    
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    
    subjectivity_score = (positive_score + negative_score) / (len(token_list)+0.000001)
    
    avg_sent_len = len(token_list)/ len(sentences)
    
    vowels = "aeiouy"
    
#     print (sentence_word_counts)
    return positive_score, negative_score, polarity_score, subjectivity_score, avg_sent_len

In [134]:
def complex_fog_avgwords_complexwords_syllable(filename):
    
    file_path = os.path.join(folder_name, filename)
    with open(file_path,'r',encoding='utf-8') as file:
        corpus = file.read()
        
    corpus = corpus.lower()
    
    corpus = re.sub('[^A-Za-z0-9]+', ' ', corpus)
#     sentences = nltk.sent_tokenize(corp)
    token_list = nltk.tokenize.word_tokenize(corpus)
    
    vowels = 'aeiou'
    syllable_per_word = []
    complex_count = 0
    for word in token_list:
        if word.endswith('es') or word.endswith('ed'):
            word = word[:-2]
        else:
            word = word
        count = 0
        for letter in word: 
            if letter in vowels:
                count+=1
            else:
                pass
        num_vowels = count
        
        syllable_per_word.append(num_vowels)
        if count>=2:
            complex_count+=1
    complex_percent = (complex_count/len(token_list))*100   
    
    a,b,c,d,avg_sent_len = pos_neg_pola_subj_avglen(filename)
    
    avg_syllable_per_word = np.mean(syllable_per_word)  
    
    fog_index = 0.4 * (avg_sent_len + complex_percent)
    
    return complex_percent, fog_index, avg_sent_len, complex_count, avg_syllable_per_word

While finding syllable_per_word, I think I need to calculate avg syllable_per_word. syllable_per_word will be a dictionary.

In [203]:
def wordcount_avgwordlen(filename):

    file_path = os.path.join(folder_name, filename)
#     print(file_path)
    with open(file_path,'r',encoding='utf-8') as file:
        corpus = file.read()
        
    corpus = corpus.lower()
    
    corpus = re.sub('[^A-Za-z0-9]+', ' ', corpus)
    token_list = nltk.tokenize.word_tokenize(corpus)
    
    total_word_length = sum(len(word) for word in token_list)
    
    stop_words = set(stopwords.words('english'))
    
    filtered_words = [word for word in token_list if word not in stop_words]
    filtered_corpus = " ".join(filtered_words)
    
    final_token_list = nltk.tokenize.word_tokenize(filtered_corpus)
    word_count = len(final_token_list)
    
    avg_word_len = total_word_length / len(token_list)
    
    return word_count, avg_word_len

In [182]:
def personal_pronoun (filename):
    
    file_path = os.path.join(folder_name, filename)
#     print(file_path)
    with open(file_path,'r',encoding='utf-8') as file:
        corpus = file.read()  
    
    token_upper = nltk.tokenize.word_tokenize(corpus)
    
    US_count = token_upper.count('US')
#     print(US_count)
    
    corpus = corpus.lower()    
    corpus = re.sub('[^A-Za-z0-9]+', ' ', corpus)

    token_list = nltk.tokenize.word_tokenize(corpus)


    target_words = ['i', 'we', 'my', 'ours', 'us']

    # Initialize a Counter to count occurrences of each target word
    word_counter = Counter()

    for word in token_list:
        if word in target_words:
            word_counter[word] += 1

    # Special handling for 'US' to avoid counting it as a pronoun
    country_word = 'us'
    word_counter[country_word] = word_counter[country_word] - US_count
    personal_pronoun_count = sum(word_counter.values())

    return personal_pronoun_count

positive_score, negative_score, polarity_score, subjectivity_score, avg_sent_len       ##1,2,3,4,5

In [201]:
positive_score = []; negative_score= []; polarity_score = []; subjectivity_score =[]; avg_sent_len =[]
for file in text_files:
    a,b,c,d,e = pos_neg_pola_subj_avglen(file)
    positive_score.append(a)
    negative_score.append(b)
    polarity_score.append(c)
    subjectivity_score.append(d)
    avg_sent_len.append(e)

complex_fog_avgwords_complexwords_syllable    ## 6,7,8,9,11

In [202]:
percent_compex_words = []; fog_index= []; avg_words_sent = []; complex_words_count =[]; syllable_per_word =[]
for file in text_files:
    f,g,h,i,k = complex_fog_avgwords_complexwords_syllable(file)
    percent_compex_words.append(a)
    fog_index.append(b)
    avg_words_sent.append(c)
    complex_words_count.append(d)
    syllable_per_word.append(e)

wordcount_avgwordlen   ##10,13


In [204]:
word_count = [] ; avg_word_len = []
for file in text_files:
    j, m = wordcount_avgwordlen(file)
    word_count.append(j)
    avg_word_len.append(m)

personal_pronoun    ##12

In [205]:
personal_pronoun_count = []
for file in text_files:
    l = personal_pronoun (file)
    personal_pronoun_count.append(l)

In [206]:
def insert_val(column):
    column.insert(7,'-'); column.insert(20,'-'); column.insert(107,'-')
    return column

In [207]:
import pandas as pd

output = pd.read_excel('Output Data Structure.xlsx')

output['POSITIVE SCORE'] = insert_val(positive_score)
output['NEGATIVE SCORE'] = insert_val(negative_score)
output['POLARITY SCORE'] = insert_val(polarity_score)
output['SUBJECTIVITY SCORE'] = insert_val(subjectivity_score)
output['AVG SENTENCE LENGTH'] = insert_val(avg_sent_len)
output['PERCENTAGE OF COMPLEX WORDS'] = insert_val(percent_compex_words)
output['FOG INDEX'] = insert_val(fog_index)
output['AVG NUMBER OF WORDS PER SENTENCE'] = insert_val(avg_words_sent)
output['COMPLEX WORD COUNT'] = insert_val(complex_words_count)
output['WORD COUNT'] = insert_val(word_count)
output['SYLLABLE PER WORD'] = insert_val(syllable_per_word)
output['PERSONAL PRONOUNS'] = insert_val(personal_pronoun_count)
output['AVG WORD LENGTH'] = insert_val(avg_word_len)


# Save the updated DataFrame back to the Excel file
output.to_excel('output_new.xlsx', index=False)